In [1]:
import pandas as pd, numpy as np
import datetime
import openpyxl

from db_utils import write_to_db  
from db_utils import fetch_data 
from utils import *
from data_manipulator import *
from utilities.connections import *

In [2]:
geo_conversions = fetch_data("SELECT id as geo_region_id , name as city FROM geo_regions",connection ="oms")
geo_conversions.loc[len(geo_conversions.index)] = [0,'India'] 
geo_conversions=geo_conversions.sort_values('geo_region_id').reset_index(drop=True)
geo_conversions.head()


Query run time in seconds : 0.7557361125946045


,geo_region_id,city
0,0,India
1,1,Mumbai
2,2,Delhi NCR
3,3,Bangalore
4,4,Hyderabad


In [3]:
q_reactivation='select * from raw_reactivation where geo_region_id >= 0 and geo_region_id <= 30'
connection='redshift'
reactivation_t=fetch_data(q_reactivation,connection)

q_retention='select * from raw_retention'
connection='redshift'
retention=fetch_data(q_retention,connection)

q_acquistion = 'select * from raw_acquisition'
connection='redshift'
acquistion=fetch_data(q_acquistion,connection)

q_lead = 'select distinct * from raw_leads'
connection='redshift'
leads=fetch_data(q_lead,connection)

Query run time in seconds : 1.4391732215881348
Query run time in seconds : 1.0806119441986084
Query run time in seconds : 1.208644151687622
Query run time in seconds : 1.0060601234436035


In [4]:
reactivation_t['geo_region_id'].unique()

array([ 1,  2,  3,  4,  5,  6,  8, 10,  9,  7, 11, 12, 13, 14, 15])

In [5]:
reactivation_t.head()

,month,geo_region_id,acc_customers,acc_g30_l60,acc_g60_l90,acc_g90_l120,acc_g120,month.1,geo_region_id.1,reactivated_customers,...,reactivated_customers_m2,reactivated_l2_m2,reactivated_g3_m2,reactivated_g30_l60_m2,reactivated_g60_l90_m2,reactivated_g90_l120_m2,reactivated_g120_m2,orders_m2,orders_l2_m2,orders_g3_m2
0,2019-01-01,1,85043,6593,7146,5028,66276,2019-01-01,1,3878,...,677.0,485.0,192.0,309.0,164.0,59.0,145.0,1541.0,646.0,895.0
1,2019-01-01,2,16132,1465,1476,1252,11939,2019-01-01,2,572,...,81.0,66.0,15.0,45.0,14.0,8.0,14.0,184.0,88.0,96.0
2,2019-01-01,3,93564,11606,10162,8336,63460,2019-01-01,3,5130,...,818.0,571.0,247.0,440.0,153.0,71.0,154.0,1981.0,769.0,1212.0
3,2019-01-01,4,23743,2363,2183,1911,17286,2019-01-01,4,870,...,142.0,108.0,34.0,71.0,27.0,14.0,30.0,327.0,149.0,178.0
4,2019-01-01,5,27363,2867,3149,2394,18953,2019-01-01,5,1268,...,227.0,158.0,69.0,116.0,42.0,22.0,47.0,508.0,210.0,298.0


In [6]:
reactivation_t.month.unique()

array(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
       '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
       '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
       '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
       '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
       '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
       '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
       '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
       '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
       '2022-05-01', '2022-06-01', '2022-07-01'], dtype=object)

In [7]:
reactivation_t['month'] = pd.to_datetime(reactivation_t['month']).dt.strftime('%Y-%m-%d')
reactivation=reactivation_t.drop(['month.1','month.2', 'month.3','geo_region_id.1','geo_region_id.2','geo_region_id.3'], axis = 1) 
reactivation=reactivation.drop_duplicates()

retention['month'] = pd.to_datetime(retention['month']).dt.strftime('%Y-%m-%d')
retention=retention.drop_duplicates()

acquistion['month'] = pd.to_datetime(acquistion['month']).dt.strftime('%Y-%m-%d')
acquistion=acquistion.drop_duplicates()

leads['month'] = pd.to_datetime(leads['month']).dt.strftime('%Y-%m-%d')
leads=leads.drop_duplicates()

In [8]:
leads=leads.fillna({'geo_region_id':20.0})

leads=leads.groupby(['month','geo_region_id'])['leads'].sum().reset_index() \
    .assign(attribution='New_Leads').append(leads,ignore_index=False)
leads=movecol(leads, 
             cols_to_move=['month','geo_region_id'], 
             ref_col='attribution',
             place='Before')


In [9]:
total_new_leads=leads.loc[leads['attribution']=='New_Leads']

In [10]:
col_list=['leads']
a=len(total_new_leads)
tot=total_new_leads.head(a)    
unique_geo_regions = [0]+tot['geo_region_id'].unique().tolist()
pan_india_totals= tot.reset_index().groupby(['month'])[col_list].sum()
pan_india_totals['geo_region_id'] = 0
pan_india_totals['attribution'] = 'New_Leads' 
pan_india_totals=pan_india_totals.reset_index()



In [11]:
pan_india_totals.to_csv('pan_india_leads.csv',index=False,header=True)

In [12]:
pan_india_totals_acq=pan_india_totals[['month',	'geo_region_id','attribution','leads']]
total_new_leads.tail()

,leads,month,geo_region_id,attribution
464,5720,2022-07-01,13.0,New_Leads
465,6329,2022-07-01,14.0,New_Leads
466,5657,2022-07-01,15.0,New_Leads
467,243,2022-07-01,16.0,New_Leads
468,25440,2022-07-01,20.0,New_Leads


In [13]:
## Appending the Pan India Leads To Total Leads

total_new_leads=total_new_leads.append(pan_india_totals_acq,ignore_index=True)

In [14]:

#pan_india_leads.to_csv('pan_india_leads.csv',index=False,header=True)
total_new_leads.to_csv('tot_leads.csv',index=False,header=True)


In [15]:
col_list_acq=acquistion.columns.values.tolist()
col_list_acq=[n for n in col_list_acq if (n != 'geo_region_id')& (n != 'month') & (n != 'attribution')]
pan_india_totals_acq=get_pan_india_totals_react(acquistion,col_list_acq)

col_list_ret=retention.columns.values.tolist()
col_list_ret= [n for n in col_list_ret if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_ret=get_pan_india_totals_react(retention,col_list_ret)

col_list_react=reactivation.columns.values.tolist()
col_list_react= [n for n in col_list_react if (n != 'geo_region_id')& (n != 'month')]
pan_india_totals_react=get_pan_india_totals_react(reactivation,col_list_react)


In [16]:
pan_india_totals_acq = movecol(pan_india_totals_acq, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_ret = movecol(pan_india_totals_ret, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')

pan_india_totals_react = movecol(pan_india_totals_react, 
             cols_to_move=['geo_region_id'], 
             ref_col='month',
             place='After')


acquistion=acquistion.append(pan_india_totals_acq,ignore_index=True)
# acquistion=pd.merge(acquistion,total_new_leads,on=['month','geo_region_id'],how='left')
# acquistion=acquistion.fillna({'attribution':'New_Leads'})

retention=retention.append(pan_india_totals_ret,ignore_index=True)
reactivation=reactivation.append(pan_india_totals_react,ignore_index=True)

In [17]:

#Merge The Acquisition DataFrame to Total Leads Data Frame
acquistion=pd.merge(acquistion,total_new_leads,how='left',on=['month','geo_region_id'])


In [18]:
acquistion=acquistion.drop_duplicates()
acquistion.to_csv('leads.csv',index=False,header=True)

In [19]:
acquistion=get_lag_columns(acquistion)
retention=get_lag_columns_ret(retention)
reactivation=get_lag_columns_react(reactivation)

In [20]:
acquistion.sort_values(by=['geo_region_id','month'], inplace=True)
acquistion=get_acquistion_data(acquistion)
acquistion=acquistion[['month',	'geo_region_id',	'customers',	'customer_less_2',	'customer_greater_3',	'orders',	'orders_less_2',	'orders_greater_3',	'customers_m_1',	'customer_less_2_m_1',	'customer_greater_3_m_1',	'orders_m_1',	'orders_less_2_m_1',	'orders_greater_3_m_1',	'customers_m_2',	'attribution',	'leads','Conversion %','order_per_customer','Order/Customer (>=3)','Zero order cust','Orders/Customer_m1','% Retention','% Customer>=3/Active']]

retention.sort_values(by=['geo_region_id','month'], inplace=True)
retention=get_retention_data(retention)

reactivation.sort_values(by=['geo_region_id','month'], inplace=True)
reactivation=get_reactivation_data(reactivation)


In [21]:
reactivation.columns

Index(['month', 'geo_region_id', 'acc_customers', 'acc_g30_l60', 'acc_g60_l90',
       'acc_g90_l120', 'acc_g120', 'reactivated_customers', 'reactivated_l2',
       'reactivated_g3', 'reactivated_g30_l60', 'reactivated_g60_l90',
       'reactivated_g90_l120', 'reactivated_g120', 'orders', 'orders_l2',
       'orders_g3', 'reactivated_customers_m1', 'reactivated_l2_m1',
       'reactivated_g3_m1', 'reactivated_g30_l60_m1', 'reactivated_g60_l90_m1',
       'reactivated_g90_l120_m1', 'reactivated_g120_m1', 'orders_m1',
       'orders_l2_m1', 'orders_g3_m1', 'reactivated_customers_m2',
       'reactivated_l2_m2', 'reactivated_g3_m2', 'reactivated_g30_l60_m2',
       'reactivated_g60_l90_m2', 'reactivated_g90_l120_m2',
       'reactivated_g120_m2', 'orders_m2', 'orders_l2_m2', 'orders_g3_m2',
       'orders_lag', 'reactivated_customers_lag', 'Orders/Customer',
       '%Customer>=3/Total Reactivated', 'percentage reactivation',
       'zero_order_customer', '% Retention', 'Orders/Customer_m1

In [22]:
## Acquisition

selected_columns = ['geo_region_id','month','Conversion %','leads','customers','customer_less_2','customer_greater_3','orders','orders_less_2',
                    'orders_greater_3','order_per_customer','Order/Customer (>=3)']
first_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','customers_m_1','customer_less_2_m_1','customer_greater_3_m_1','customers_m_2','Zero order cust','orders_m_1',
                    'orders_less_2_m_1','orders_greater_3_m_1','Orders/Customer_m1']
second_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

selected_columns =['geo_region_id','month','% Retention','% Customer>=3/Active']
third_df_dict_acq=get_data_subset_acq(selected_columns,acquistion)

## Reactivation
selected_columns =['geo_region_id','month','acc_customers','acc_g30_l60','acc_g60_l90','acc_g90_l120','acc_g120']
first_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)

selected_columns = ['geo_region_id','month','reactivated_customers','reactivated_l2','reactivated_g3','orders',
                        'orders_l2','orders_g3','Orders/Customer']
second_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','percentage reactivation','%Customer>=3/Total Reactivated','% Retention','reactivated_g30_l60','reactivated_g60_l90','reactivated_g90_l120','reactivated_g120']
third_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns = ['geo_region_id','month','reactivated_customers_m1','reactivated_l2_m1','reactivated_g3_m1',
                        'reactivated_customers_m2','zero_order_customer']
fourth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)
    
selected_columns =['geo_region_id','month','orders_m1','orders_l2_m1','orders_g3_m1','reactivated_g30_l60_m1','reactivated_g60_l90_m1',
                       'reactivated_g90_l120_m1']
fifth_df_dict_reactivation = get_data_subset_react(selected_columns, reactivation)


## Retention 
selected_columns = ['geo_region_id','month','customers_retained','less_than_two_order_customers','greater_than_two_order_customers',]
first_df_dict = get_data_subset(selected_columns, retention)
    
selected_columns = ['geo_region_id','month','total_orders','orders_less_than_2','orders_greater_than_2','order_per_customer']
second_df_dict = get_data_subset(selected_columns, retention)

selected_columns = ['geo_region_id','month','m1_customers_retained','m1_less_than_two_order_customers','m1_greater_than_two_order_customers',
                    'm1_retention_history','zero_order_customer','M-1_order_per_customer','%_Retention']
third_df_dict = get_data_subset(selected_columns, retention)



In [23]:
[0]+sorted(reactivation_t['geo_region_id'].unique().tolist())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [24]:
df_dict = {}
#for geo_region in [0,1,2,3,4,5,6,8,10]:
for geo_region in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    df_dict[str(geo_region)] = {
        'Acquisition Customers & Orders':acquisition_first_metric(first_df_dict_acq[geo_region]),
        'Retention for Acquisition Base':acquisition_second_metric(second_df_dict_acq[geo_region]),
        'Retention % for Acquistion Base':acquisition_third_metric(third_df_dict_acq[geo_region]),
        'Retained Customer Base':retention_first_metric(first_df_dict[geo_region]),
        'Retained Customers Orders':retention_second_metric(second_df_dict[geo_region]),
        'M-1 Retention' :retention_last_month_metric(third_df_dict[geo_region]),
        'Accumulated Base':reactivation_first_metric(first_df_dict_reactivation[geo_region]),
        'Reactivated Customers':reactivation_customer_metric(second_df_dict_reactivation[geo_region]),
        'Reactivation Base':reactivation_days_metric(third_df_dict_reactivation[geo_region]),
        'Retention for Reactivation Base':reactivation_second_metric(fourth_df_dict_reactivation[geo_region]),
        'NextMonth Retention for Reactivation Base':reactivation_third_metric(fifth_df_dict_reactivation[geo_region])
        
        
    }        

In [25]:
# for key in df_dict['7'].keys():
#     df_dict['7'][key].fillna(0, inplace=True)

In [25]:
df_dict['11']['Acquisition Customers & Orders']

month,2021-07-01,2021-08-01,2021-09-01,2021-10-01,2021-11-01,2021-12-01,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01
Conversion %,4.84,7.79,8.44,12.50,14.22,12.22,11.37,15.91,18.53,18.97,20.20,21.04,14.85
New_Leads,6132.00,6250.00,7239.00,8806.00,7208.00,9763.00,10859.00,9848.00,8933.00,9638.00,11168.00,10623.00,13041.00
Customer,297.00,487.00,611.00,1101.00,1025.00,1193.00,1235.00,1567.00,1655.00,1828.00,2256.00,2235.00,1936.00
Customers <=2,282.00,446.00,544.00,932.00,858.00,1006.00,1152.00,1456.00,1532.00,1707.00,2097.00,2090.00,1806.00
Customers >=3,15.00,41.00,67.00,169.00,167.00,187.00,83.00,111.00,123.00,121.00,159.00,145.00,130.00
Order,390.00,645.00,897.00,1852.00,1795.00,1998.00,1703.00,2134.00,2277.00,2486.00,3102.00,2959.00,2638.00
Order <=2,312.00,488.00,617.00,1104.00,1039.00,1193.00,1336.00,1671.00,1748.00,1892.00,2326.00,2331.00,2058.00
Order >=3,78.00,157.00,280.00,748.00,756.00,805.00,367.00,463.00,529.00,594.00,776.00,628.00,580.00
order_per_customer,1.31,1.32,1.47,1.68,1.75,1.67,1.38,1.36,1.38,1.36,1.38,1.32,1.36
Order/Customer (>=3),5.20,3.83,4.18,4.43,4.53,4.30,4.42,4.17,4.30,4.91,4.88,4.33,4.46


In [26]:
for key in df_dict['7'].keys():
    df_dict['7'][key]= df_dict['7'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['7']['Acquisition Customers & Orders'].columns))]

In [27]:
for key in df_dict['11'].keys():
    df_dict['11'][key]= df_dict['11'][key][list( filter(lambda x: x>= '2021-06-01', df_dict['11']['Acquisition Customers & Orders'].columns))]

In [28]:
    def generate_excel_workbook():
        """ Creates an excel sheet that has necessary information """
        #geo_regions = [0,1,2,3,4,5,6,8,10]
        geo_regions = [0]+sorted(reactivation_t['geo_region_id'].unique().tolist())
        data_frame_dicts = {}
        for geo_region in geo_regions:
            dataframe_array = []
            for table in df_dict[str(geo_region)]:
                write_to_excel = df_dict[str(geo_region)][table].reset_index()
                write_to_excel.rename(columns = {'index' :table}, inplace = True)
                dataframe_array.append(write_to_excel)
            sheet_name = "geo_region_id_" + str(geo_region)
            data_frame_dicts[sheet_name] = dataframe_array
        run_time = (str(datetime.datetime.now()).split(" ")[1].split(".")[0]).replace(":","_")
        sheet_name = "./demand_report_workbook" + ".xlsx" 
        writer = pd.ExcelWriter(sheet_name,engine ='xlsxwriter') 
        workbook  = writer.book
        bold = workbook.add_format({'bold': True})
#         for key in data_frame_dicts.keys():
#             row = 0
#             worksheet.write(row, col, key)
#             row += 1 


        for geo_region, data in data_frame_dicts.items():
            print(data)
            col = 1
            row = 1
            for df in data :
                print("hello")
                add_col = len(df.columns)
                add_row = len(df.index)
                df.to_excel(writer, sheet_name = str(geo_region), 
                     startrow = row, startcol = col, index = False)
                worksheet = writer.sheets[geo_region]
                header_format = workbook.add_format({
                              'bold': True,
                              'text_wrap': True,
                              'valign': 'top',
                              'fg_color': '#003366',
                              'font_color': 'white',
                              'border': 1,
                              'num_format': 'mmm-yy' })
               
                for f, b in zip(df.columns.values, list(range(col, col + add_col))):
                    worksheet.write(row, b , f, header_format)
                row = row + add_row + 4
        writer.save()


In [29]:
generate_excel_workbook()

[month Acquisition Customers & Orders  2019-01-01  2019-02-01  2019-03-01  \
0                       Conversion %       38.98       38.76       35.46   
1                          New_Leads    47633.00    52563.00    71766.00   
2                           Customer    18568.00    20372.00    25446.00   
3                      Customers <=2    16896.00    18672.00    23388.00   
4                      Customers >=3     1672.00     1700.00     2058.00   
5                              Order    26858.00    29031.00    35876.00   
6                          Order <=2    19264.00    21308.00    26692.00   
7                          Order >=3     7594.00     7723.00     9184.00   
8                 order_per_customer        1.45        1.43        1.41   
9               Order/Customer (>=3)        4.54        4.54        4.46   

month  2019-04-01  2019-05-01  2019-06-01  2019-07-01  2019-08-01  2019-09-01  \
0           29.66       33.49       32.31       25.31       26.14       28.57   


[6 rows x 44 columns]]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2019-01-01  2019-02-01  2019-03-01  \
0                       Conversion %       50.15       51.73       46.59   
1                          New_Leads    18141.00    19070.00    26390.00   
2                           Customer     9097.00     9865.00    12296.00   
3                      Customers <=2     8386.00     9108.00    11409.00   
4                      Customers >=3      711.00      757.00      887.00   
5                              Order    12638.00    13681.00    16747.00   
6                          Order <=2     9557.00    10342.00    12930.00   
7                          Order >=3     3081.00     3339.00     3817.00   
8                 order_per_customer        1.39        1.39        1.36   
9               Order/Customer (>=3)        4.33        4.41        4.30   

month  2019-04-01  2019-05-01  2019-06-01  2019-07-01  2019-08-01  2019-0

[month Acquisition Customers & Orders  2019-01-01  2019-02-01  2019-03-01  \
0                       Conversion %       35.49       41.23       39.24   
1                          New_Leads     5402.00     5881.00     8579.00   
2                           Customer     1917.00     2425.00     3366.00   
3                      Customers <=2     1706.00     2233.00     3130.00   
4                      Customers >=3      211.00      192.00      236.00   
5                              Order     2828.00     3297.00     4673.00   
6                          Order <=2     1909.00     2531.00     3544.00   
7                          Order >=3      919.00      766.00     1129.00   
8                 order_per_customer        1.48        1.36        1.39   
9               Order/Customer (>=3)        4.36        3.99        4.78   

month  2019-04-01  2019-05-01  2019-06-01  2019-07-01  2019-08-01  2019-09-01  \
0           31.49       34.55       32.83       26.37       28.16       30.83   


[month Acquisition Customers & Orders  2019-01-01  2019-02-01  2019-03-01  \
0                       Conversion %       19.89       21.57       18.12   
1                          New_Leads     1664.00     1507.00     1518.00   
2                           Customer      331.00      325.00      275.00   
3                      Customers <=2      283.00      286.00      252.00   
4                      Customers >=3       48.00       39.00       23.00   
5                              Order      628.00      743.00      445.00   
6                          Order <=2      348.00      389.00      333.00   
7                          Order >=3      280.00      354.00      112.00   
8                 order_per_customer        1.90        2.29        1.62   
9               Order/Customer (>=3)        5.83        9.08        4.87   

month  2019-04-01  2019-05-01  2019-06-01  2019-07-01  2019-08-01  2019-09-01  \
0           15.44       14.03       17.27       14.62       15.74       12.56   


[6 rows x 24 columns]]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-03-01  2021-04-01  2021-05-01  \
0                       Conversion %        6.81        4.18        4.69   
1                          New_Leads     2731.00     9688.00     5396.00   
2                           Customer      186.00      405.00      253.00   
3                      Customers <=2      177.00      371.00      230.00   
4                      Customers >=3        9.00       34.00       23.00   
5                              Order      239.00      526.00      358.00   
6                          Order <=2      196.00      403.00      265.00   
7                          Order >=3       43.00      123.00       93.00   
8                 order_per_customer        1.28        1.30        1.42   
9               Order/Customer (>=3)        4.78        3.62        4.04   

month  2021-06-01  2021-07-01  2021-08-01  2021-09-01  2021-10-01  2021-1

hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-07-01  2021-08-01  2021-09-01  \
0                       Conversion %        4.84        7.79        8.44   
1                          New_Leads     6132.00     6250.00     7239.00   
2                           Customer      297.00      487.00      611.00   
3                      Customers <=2      282.00      446.00      544.00   
4                      Customers >=3       15.00       41.00       67.00   
5                              Order      390.00      645.00      897.00   
6                          Order <=2      312.00      488.00      617.00   
7                          Order >=3       78.00      157.00      280.00   
8                 order_per_customer        1.31        1.32        1.47   
9               Order/Customer (>=3)        5.20        3.83        4.18   

month  2021-10-01  2021-11-01  2021-12-01  2022-01-01  2022-02-01  2022-03-01  \
0           12.50       14.22       12.22       11.37    

5             1.0  ]
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
[month Acquisition Customers & Orders  2021-12-01  2022-01-01  2022-02-01  \
0                       Conversion %        9.85       12.67       19.95   
1                          New_Leads      964.00     2897.00     3674.00   
2                           Customer       95.00      367.00      733.00   
3                      Customers <=2       86.00      264.00      444.00   
4                      Customers >=3        9.00      103.00      289.00   
5                              Order      145.00      716.00     1822.00   
6                          Order <=2      106.00      323.00      570.00   
7                          Order >=3       39.00      393.00     1252.00   
8                 order_per_customer        1.53        1.95        2.49   
9               Order/Customer (>=3)        4.33        3.82        4.33   

month  2022-03-01  2022-04-01  2022-05-01  2022-06-01  2022-07-01  
0      

## Share the Below Generated Excel Report  - demand_report_workbook.xlsx

In [30]:
### Extra Totals | Formatting & Designing the Sheet(s) & Workbook | Styling the Workbook 
## Formatting & Designing the Workbook

from openpyxl import load_workbook
from openpyxl import Workbook
wb = openpyxl.load_workbook('demand_report_workbook.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
for sheet in sheets:    
    ##Fixing the Width of the Metric Header Column
    wb[sheet].column_dimensions['B'].width = 36    
    ## Freezing the Window Pane
    wb[sheet].freeze_panes = 'C2'    
    ## Removing the Grid Lines
    wb[sheet].sheet_view.showGridLines = False
    
wb.save('demand_report_workbook.xlsx')     

## Getting the Grand Total of Orders for Pan India & All City

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook.xlsx')
sheet=wb.active
i=sheet.max_row+4
sheets=wb.sheetnames[0:len(wb.sheetnames)]
#sheets=[n for n in sheets if (n != 'Pune')]
for sheet in sheets:
    print(sheet)
    for j in range(3,wb[sheet].max_column+1):
        ws=wb[sheet]
        wb[sheet].cell(row=i,column=2).value='Total Customers'
        ws.cell(row=i,column=j).value=(ws.cell(row=5,column=j).value)+(ws.cell(row=36,column=j).value)+(ws.cell(row=71,column=j).value)
        wb[sheet].cell(row=i+1,column=2).value='Total Orders'
        wb[sheet].cell(row=i+1,column=j).value=(wb[sheet].cell(row=8,column=j).value)+(wb[sheet].cell(row=43,column=j).value)+(wb[sheet].cell(row=74,column=j).value)  
                
wb.save("demand_report_workbook.xlsx")  

## Workbook styling
## Border Styling & Beautification the Grand Totals

from openpyxl.styles.borders import Border, Side
from openpyxl import Workbook

dash_border = Border(left=Side(style='mediumDashDot'), 
                     right=Side(style='mediumDashDot'), 
                     top=Side(style='mediumDashDot'), 
                     bottom=Side(style='mediumDashDot'))

from openpyxl.formula.translate import Translator
wb = openpyxl.load_workbook('demand_report_workbook.xlsx')
sheet=wb.active
sheets=wb.sheetnames[0:len(wb.sheetnames)]
sheets
for sheet in sheets:
    wb[sheet].cell(row=111,column=2).border = dash_border
    wb[sheet].cell(row=112,column=2).border = dash_border        
    
wb.save("demand_report_workbook.xlsx")  

## Coloring & Naming the Sheets to City & Individual Tab 
## Naming the Sheets
from openpyxl import load_workbook
from openpyxl import Workbook
wb=openpyxl.load_workbook('demand_report_workbook.xlsx') 

for i in [0]+sorted(reactivation_t['geo_region_id'].unique().tolist()):
    try:
        indx = (geo_conversions[geo_conversions['geo_region_id']==i]).index[0]
        wb['geo_region_id_{a}'.format(a=i)].title = geo_conversions.iloc[indx]['city']
    except Exception:
        pass    


# ws1=wb['geo_region_id_0'].title='India'
# ws2=wb['geo_region_id_1'].title='Mumbai'
# ws3=wb['geo_region_id_2'].title='Delhi'
# ws4=wb['geo_region_id_3'].title='Bangalore'
# ws5=wb['geo_region_id_4'].title='Hyderabad'
# ws6=wb['geo_region_id_5'].title='Chennai'
# ws7=wb['geo_region_id_6'].title='Ahmedabad'
# ws8=wb['geo_region_id_8'].title='Pune'
# ws9=wb['geo_region_id_10'].title='Surat'

wb.save('demand_report_workbook.xlsx') 


## India Flag Pattern
color_map={0:'00FF6600',1:'00FFFFFF',2:'0000FF00'}
wb = openpyxl.load_workbook('demand_report_workbook.xlsx')
# for sheet in wb.worksheets:

for index,name in enumerate(wb.sheetnames):
    color = color_map[index%3]
    wb[name].sheet_properties.tabColor = color



#     wb['India'].sheet_properties.tabColor = "00FF6600"
#     wb['Mumbai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Delhi'].sheet_properties.tabColor = "0000FF00"
#     wb['Bangalore'].sheet_properties.tabColor = "00FF6600"
#     wb['Chennai'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Hyderabad'].sheet_properties.tabColor = "0000FF00"
#     wb['Ahmedabad'].sheet_properties.tabColor = "00FF6600"
#     wb['Pune'].sheet_properties.tabColor = "00FFFFFF"
#     wb['Surat'].sheet_properties.tabColor = "0000FF00"

wb.save("demand_report_workbook.xlsx")




geo_region_id_0
geo_region_id_1
geo_region_id_2
geo_region_id_3
geo_region_id_4
geo_region_id_5
geo_region_id_6
geo_region_id_7
geo_region_id_8
geo_region_id_9
geo_region_id_10
geo_region_id_11
geo_region_id_12
geo_region_id_13
geo_region_id_14
geo_region_id_15


In [33]:
sheets

['geo_region_id_0',
 'geo_region_id_1',
 'geo_region_id_2',
 'geo_region_id_3',
 'geo_region_id_4',
 'geo_region_id_5',
 'geo_region_id_6',
 'geo_region_id_7',
 'geo_region_id_8',
 'geo_region_id_9',
 'geo_region_id_10',
 'geo_region_id_11',
 'geo_region_id_12',
 'geo_region_id_13',
 'geo_region_id_14',
 'geo_region_id_15']